# 1. Web scraping using BeautifulSoup

In [1]:
import pandas as pd # Library for data analysis
from bs4 import BeautifulSoup #Library for webpage scraping
import urllib.request # Library for opening URLs

<p><b>Specify URL for the webpage and retrieve page data</b></p>

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pagedata = urllib.request.urlopen(url)

<p><b>Use BeautifulSoup for parsing HTML/XML of pagesource</b></p>

In [3]:
soup = BeautifulSoup(pagedata,'lxml')

In [4]:
#title for table
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [8]:
#find table data 
table_data = soup.find('table',class_="wikitable")

In [9]:
#Make 3 lists to retrieve data for 3 columns-Postal Code,Borough,Neighborhood
list_PC=[]
list_Borough = []
list_Neighborhood = []

#Loop the table_data and fill in all the list with appropriate data.
for row in table_data.findAll('tr'):
    td_row=row.findAll('td',text=True)
    #print(len(td_row))
    if (len(td_row)==3):
        list_PC.append(td_row[0].find(text=True))
        list_Borough.append(td_row[1].find(text=True))
        list_Neighborhood.append(td_row[2].find(text=True))
    

In [11]:
#Make DataFrame using Pandas
df1 = pd.DataFrame({'Postal Code':list_PC,'Borough':list_Borough,'Neighborhood':list_Neighborhood})
df1.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [12]:
#Formatting string data to remove unnecessary '\n' character
df1['Postal Code']=df1['Postal Code'].apply(lambda x: x.strip())
df1['Borough']=df1['Borough'].apply(lambda x: x.strip())
df1['Neighborhood']=df1['Neighborhood'].apply(lambda x: x.strip())

In [15]:
df1.loc[df1['Borough'] == 'Not assigned'].count()

Postal Code     77
Borough         77
Neighborhood    77
dtype: int64

In [16]:
df1.shape

(179, 3)

<p> Removing rows with Borough = 'Not assigned'

In [19]:
#df1-original dataframe containing data from Webpage 
#df2-copy of original dataframe to be modified  

df2=df1.copy() #create deep copy of original dataframe df1

#Removed rows with Borough = 'Not assigned' and make index adjustment

df2.drop(df2[df2['Borough']=='Not assigned'].index,inplace=True)
df2.reset_index(drop=True,inplace=True)

df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [20]:
df2.shape

(102, 3)

<p> If a borough has neighborhood value as 'Not assigned' then Neighborhood value should be same as Borough</p>

In [26]:
for index,row in df2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] == row['Borough']


<p> If for a Postal code and Borough combination, multiple Neighborhoods exist, then combine them using ','.</p>


In [27]:
df2.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


As we can see neighborhoods for a group of Postal code and Borough were already combined using '/' on wikipedia page itself, I will just replace the '/' with ',' here.     

In [28]:
#Separate Neighborhoods by comma 
df2.replace(to_replace = {'Neighborhood': '/'},value = ',',regex= True,inplace=True)

df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [29]:
df2.shape

(102, 3)

# 2. Exploring neighborhoods in Toronto

In [30]:
#import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


<p><b>Random check if geocoder is able to get correct values of coordinates for postal codes</b></p>

In [31]:
#Coordinates given by this package don't seem to be correct and doesn't give coordinates for few other postal codes
address = 'M4B'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of address are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of address are 45.4405689, 28.019180614730512.


In [32]:
#As already mentioned in the assignment instructions, this module doesn't seem to be reliable

#!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
g = geocoder.google('M4M, Toronto, Ontario')
lat_lng_coords = g.latlng

print(lat_lng_coords)


None


In [33]:
#Getting coordinates from csv file link provided in the instructions
df3= pd.read_csv('http://cocl.us/Geospatial_data')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df3.shape

(103, 3)

<p><b>Final dataframe with all postal codes,Boroughs,Neighborhoods and coordinate values of Toronto Neighborhoods</b></p>

In [34]:
#df4- final dataframe with latitude and longitude values 
df4=pd.merge(df2,df3,how='left',on=['Postal Code'])
df4.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


<p><b>Get coordinates of Toronto to create map</b></p>

In [35]:
address = 'Toronto,Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of address are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of address are 43.6534817, -79.3839347.


<p><b>Create map of Toronto with neighborhoods superimposed on top</b></p>

In [36]:
#Create map of toronto with coordinates values
toronto_map=folium.Map(location=[latitude,longitude])

for lat,long,borough,neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
        location=[lat,long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 

toronto_map   

<p><b>Get coordinates of boroughs having word 'Toronto', and plot map for those boroughs</b></p>

In [37]:
#get coordinates of boroughs having word 'Toronto', and plot map for those boroughs
df5=df4[df4['Borough'].str.contains('Toronto')].reset_index(drop=True)

toronto2_map = folium.Map(location=[latitude,longitude],zoom_start=10) 
for lat,long,borough,neighborhood in zip(df5['Latitude'],df5['Longitude'],df5['Borough'],df5['Neighborhood']):
    
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    
    folium.CircleMarker(
    location=[lat,long],
    popup = label,
    radius = 5,
    color = 'blue',
    fill = True,
    fill_color ='#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(toronto2_map)

toronto2_map

<p><b>Using Foursquare API to explore nearby venues for neighborhoods in Toronto</b></p>

In [38]:
CLIENT_ID = 'WTYTUTTJ2DPSFHAS4KY03TSYRZZS0JFJ1X0ST42NJUEGJWT4' # your Foursquare ID
CLIENT_SECRET = 'W4FPORRDHKEZI3F0CJVOEMIIFARGUU4XZCO3OUX3KSFQNNBU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: WTYTUTTJ2DPSFHAS4KY03TSYRZZS0JFJ1X0ST42NJUEGJWT4
CLIENT_SECRET:W4FPORRDHKEZI3F0CJVOEMIIFARGUU4XZCO3OUX3KSFQNNBU


<p><b>Explore 1st neighborhood from dataset df5</b></p>

In [39]:
df5.loc[0,'Neighborhood']#name of 1st Neighborhood having borough containing word Toronto

'Regent Park , Harbourfront'

In [40]:
#Get coordinates of Regent Park,Harbourfront
neighborhood_lat=df5.loc[0,'Latitude']
neighborhood_long=df5.loc[0,'Longitude']
neighborhood_name=df5.loc[0,'Neighborhood']

print("Coordinates of {} are {},{}".format(neighborhood_name,neighborhood_lat,neighborhood_long))

Coordinates of Regent Park , Harbourfront are 43.6542599,-79.3606359


<p><b>Get top 100 venues nearby Regent Park,Harbourfront</b></p>

In [41]:
#parameters for url
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    LIMIT)

#send GET request and examine results
results = requests.get(url).json()
#results
#results['response']['groups'][0]['items'][0]['venue']['name']--o/p:Roselle Desserts
#results['response']['groups'][0]['items'][1]['venue']['name']--o/p:Tandem Coffee

In [42]:
#Function:To get category name from categories/venue categories 

def get_category_name(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues= results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

#select columns 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_name,axis=1)
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [44]:
#clean column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


<p><b> Get dataset for venues near all the neighborhoods of dataframe df5 </b></p>

In [43]:
#function to retrieve values from result and create dataframe out of it

def get_Nearby_Venues(postal_code,borough,neighborhood,neighborhood_lat,neighborhood_long,radius = 500):
    
    venues_list=[]
    for pc,br,nhood,nhood_lat,nhood_long in zip(postal_code,borough,neighborhood,neighborhood_lat,neighborhood_long):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            nhood_lat, 
            nhood_long, 
            radius, 
            LIMIT)

        #send GET request and examine results
        results = requests.get(url).json()["response"]['groups'][0]['items']
        

        for r in results:
            venues_list.append(
                [pc,
                br,
                nhood,
                nhood_lat,
                nhood_long,
                r['venue']['name'],
                r['venue']['location']['lat'],
                r['venue']['location']['lng'],
                r['venue']['categories'][0]['name']])
    
    #print(venues_list[0])
    
    df6_venues = pd.DataFrame() #Create empty dataframe object
    
    #create blank lists for each column data
    list_pc=[]
    list_br=[]
    list_nhood=[]
    list_nhood_lat=[]
    list_nhood_lng=[]
    list_venue=[]
    list_venue_lat=[]
    list_venue_lng=[]
    list_category=[]
    
    for i in range(len(venues_list)):
        
        list_pc.append(venues_list[i][0])
        list_br.append(venues_list[i][1])
        list_nhood.append(venues_list[i][2])
        list_nhood_lat.append(venues_list[i][3])
        list_nhood_lng.append(venues_list[i][4])
        list_venue.append(venues_list[i][5])
        list_venue_lat.append(venues_list[i][6])
        list_venue_lng.append(venues_list[i][7])
        list_category.append(venues_list[i][8])
    
    df6_venues['Postal Code'] = list_pc
    df6_venues['Borough'] = list_br
    df6_venues['Neighborhood'] = list_nhood
    df6_venues['Neighborhood Latitude'] = list_nhood_lat
    df6_venues['Neighborhood Longitude'] = list_nhood_lng
    df6_venues['Venue name'] = list_venue
    df6_venues['Venue Latitude'] = list_venue_lat
    df6_venues['Venue Longitude'] = list_venue_lng
    df6_venues['Venue Category'] = list_category
    
    return(df6_venues)

In [44]:
Toronto_venues = get_Nearby_Venues(postal_code=df5['Postal Code'],
                  borough=df5['Borough'],
                  neighborhood=df5['Neighborhood'],
                  neighborhood_lat=df5['Latitude'],
                  neighborhood_long=df5['Longitude'])

Toronto_venues.shape

(1667, 9)

In [45]:
Toronto_venues.head()

,Postal Code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue name,Venue Latitude,Venue Longitude,Venue Category
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [47]:
#Venues for each neighborhood
Toronto_venues.groupby('Neighborhood').count()['Venue name']

Neighborhood
Berczy Park                                                               57
Brockton , Parkdale Village , Exhibition Place                            23
Business reply mail Processing CentrE                                     18
Central Bay Street                                                        76
Christie                                                                  17
Church and Wellesley                                                      79
Commerce Court , Victoria Hotel                                          100
Davisville                                                                38
Davisville North                                                           6
Dufferin , Dovercourt Village                                             16
First Canadian Place , Underground city                                  100
Forest Hill North & West                                                   4
Garden District, Ryerson                                       

In [48]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 225 uniques categories.


# 3. Analyze each neighborhood

In [56]:
Toronto_venues_onehot.drop('Neighborhood',axis=1,inplace=True)
Toronto_venues_onehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [61]:
# one hot encoding
Toronto_venues_onehot = pd.get_dummies(data = Toronto_venues['Venue Category'],prefix="",prefix_sep = "")
#Toronto_venues_onehot.insert(0,'Neighborhood',Toronto_venues['Neighborhood'])
#Toronto_venues_onehot.drop('Neighborhood',axis=1,inplace=True)
Toronto_venues_onehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [62]:
#Add column Neighborhood at start
Toronto_venues_onehot.insert(0,'Neighborhood',Toronto_venues['Neighborhood'])
Toronto_venues_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [63]:
Toronto_venues_onehot.shape

(1667, 225)

<p><b> Mean of frequency of occurence for each category group by neighborhood</b></p>

In [64]:
Toronto_venues_onehot_grouped = Toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_venues_onehot_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.017544,0.000000,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.000000,0.0,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017544,0.035088,0.0,0.0,0.000000,0.0,0.035088,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.105263,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.017544,0.0,0.017544,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0

<b>Print each neighborhood with top 5 most common venues</b>

In [65]:
num_top_venues =5

for hood in Toronto_venues_onehot_grouped['Neighborhood']:
    print("----"+hood+"----")
    #get transpose using T
    temp = Toronto_venues_onehot_grouped[Toronto_venues_onehot_grouped['Neighborhood']==hood].T.reset_index()
    temp=temp.iloc[1:] #1st row will be Neighborhood and Neighborhood value, so ignore 1st row
    temp.columns = ['venue category','frequency']
    temp['frequency']=temp['frequency'].astype(float)
    temp = temp.round({'frequency': 2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
       venue category  frequency
0         Coffee Shop       0.11
1        Cocktail Bar       0.05
2  Seafood Restaurant       0.04
3         Cheese Shop       0.04
4          Restaurant       0.04


----Brockton , Parkdale Village , Exhibition Place----
      venue category  frequency
0               Café       0.13
1        Coffee Shop       0.09
2     Breakfast Spot       0.09
3        Yoga Studio       0.04
4  Convenience Store       0.04


----Business reply mail Processing CentrE----
       venue category  frequency
0  Light Rail Station       0.11
1         Yoga Studio       0.06
2          Comic Shop       0.06
3    Recording Studio       0.06
4          Restaurant       0.06


----Central Bay Street----
        venue category  frequency
0          Coffee Shop       0.18
1   Italian Restaurant       0.05
2                 Café       0.05
3  Japanese Restaurant       0.04
4       Sandwich Place       0.04


----Christie----
  venue category  frequency
0  Groc

           venue category  frequency
0        Greek Restaurant       0.19
1             Coffee Shop       0.10
2      Italian Restaurant       0.07
3          Ice Cream Shop       0.05
4  Furniture / Home Store       0.05


----Toronto Dominion Centre , Design Exchange----
       venue category  frequency
0         Coffee Shop       0.15
1                Café       0.07
2               Hotel       0.07
3          Restaurant       0.05
4  Italian Restaurant       0.03


----University of Toronto , Harbord----
        venue category  frequency
0                 Café       0.14
1  Japanese Restaurant       0.06
2                  Bar       0.06
3            Bookstore       0.06
4               Bakery       0.06




In [66]:
Toronto_venues_onehot_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.017544,0.000000,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.000000,0.0,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017544,0.035088,0.0,0.0,0.000000,0.0,0.035088,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.105263,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.017544,0.0,0.017544,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0

<b>Create a dataframe showing 10 most popular venue categories against each neighborhood</b>

In [67]:
num_top = 10
#create column list

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Toronto_most_pop_categories = pd.DataFrame(columns = columns)
Toronto_most_pop_categories['Neighborhood']=Toronto_venues_onehot_grouped['Neighborhood']



In [68]:
for ind in np.arange(Toronto_venues_onehot_grouped.shape[0]):
    
    row_data = Toronto_venues_onehot_grouped.iloc[ind,1:]
    row_data_sorted = row_data.sort_values(ascending = False)
    Toronto_most_pop_categories.iloc[ind,1:]=row_data_sorted.index.values[0:num_top]

Toronto_most_pop_categories.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Café,Beer Bar,Farmers Market,Cheese Shop,Shopping Mall
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Gym,Furniture / Home Store,Intersection,Italian Restaurant,Nightclub
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Garden Center,Fast Food Restaurant,Skate Park,Smoke Shop,Spa,Brewery,Restaurant,Recording Studio
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Department Store,Dessert Shop,Bubble Tea Shop,Thai Restaurant
4,Christie,Grocery Store,Café,Park,Gas Station,Baby Store,Italian Restaurant,Diner,Coffee Shop,Nightclub,Restaurant


# 4.Clustering Neighborhoods

In [69]:
#set number of clusters
kclusters = 3

Toronto_grouped_cluster = Toronto_venues_onehot_grouped.drop('Neighborhood',1)

#kmeans clustering
kmeans = KMeans(n_clusters=kclusters,random_state=0)
kmeans.fit(Toronto_grouped_cluster)

#kmeans labels generated for each row in Toronto_grouped_cluster dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [71]:
#Create a dataframe to show cluster lables as well as top 10 most popular venue categories for that neighborhood
Toronto_most_pop_categories.insert(0,'Cluster labels',kmeans.labels_)

Toronto_merged = df5
Toronto_merged = Toronto_merged.join(Toronto_most_pop_categories.set_index('Neighborhood'),on='Neighborhood')

Toronto_merged.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Mexican Restaurant,Theater,Restaurant,Café,Breakfast Spot,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Café,Fried Chicken Joint,Italian Restaurant,Boutique,Juice Bar,Burger Joint,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Theater,Fast Food Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,American Restaurant,Diner,Gastropub,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [72]:
#Visualizing clusters

cluster_map=folium.Map(location=[latitude,longitude],zoom_start=10) 

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
#rainbow=['#CD5C5C','#FF00FF','#32CD32','#1E90FF','#191970']
rainbow=['#CD5C5C','#FF00FF','#32CD32']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#        color=rainbow[cluster-1],
        color=rainbow[cluster],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

<b>Cluster 1 with cluster label = 0 </b>
<p> This cluster mainly contains coffee shops,grocery stores,restaurents and shopping places as popular venues.</p>

In [73]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Mexican Restaurant,Theater,Restaurant,Café,Breakfast Spot,Chocolate Shop
1,Downtown Toronto,0,Coffee Shop,Diner,Yoga Studio,Café,Fried Chicken Joint,Italian Restaurant,Boutique,Juice Bar,Burger Joint,Burrito Place
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Theater,Fast Food Restaurant,Diner
3,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,American Restaurant,Diner,Gastropub,Beer Bar
4,East Toronto,0,Health Food Store,Pub,Trail,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Café,Beer Bar,Farmers Market,Cheese Shop,Shopping Mall
6,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Department Store,Dessert Shop,Bubble Tea Shop,Thai Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Gas Station,Baby Store,Italian Restaurant,Diner,Coffee Shop,Nightclub,Restaurant
8,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Gym,Thai Restaurant,Bakery,Lounge,Cosmetics Shop,Concert Hall,Office
9,West Toronto,0,Pharmacy,Bakery,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Pizza Place,Recording Studio,Café,Brewery


<b>Cluster 2 with cluster label = 1 </b>
<p>This cluster does not have coffee shops and shopping places like cluster 1 but has playground and fitness center as most popular places.</p>

In [74]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Playground,Gym / Fitness Center,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


<b>Cluster 3 with cluster label = 2 </b>
<p>This cluster does not have coffee shops and shopping places like cluster 1 but has parks as most popular venue</p>

In [75]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
21,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
32,Downtown Toronto,2,Park,Playground,Trail,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
